In [11]:
import pandas as pd
import plotly.express as px

# Load your data
df = pd.read_csv('NoAeroRun.csv')

# Convert the times to milliseconds
start_time_ms = 9 * 60 * 60 * 1000 + 12 * 60 * 1000 + 33 * 1000 + 5
end_time_ms = 9 * 60 * 60 * 1000 + 13 * 60 * 1000 + 59 * 1000 + 2

# Indices and their corresponding times
start_index = 2121
end_index = 13263

# Time difference and index difference
time_diff_ms = end_time_ms - start_time_ms
index_diff = end_index - start_index

# Time per index
time_per_index_ms = time_diff_ms / index_diff

# Define the radius of the tires
radius = 0.26035  # in meters
gear_ratio = 3.54

# Convert motor speed (RPM) to wheel speed (m/s)
df['Wheel_Speed'] = (df['D2_Motor_Speed'] * (2 * 3.141592653589793 * radius) / 60) / gear_ratio

# Invert and convert IVT_Result_I to kiloamperes
df['IVT_Result_I'] = df['IVT_Result_I'] * -1 / 1000

# Define the time interval in seconds (9 milliseconds)
time_interval = 0.009  # seconds

# Calculate power at each time point
df['Power'] = df['D1_DC_Bus_Voltage'] * df['IVT_Result_I']

# Calculate incremental energy used at each time point
df['Incremental_Energy'] = df['Power'] * time_interval  # Energy in Joules (Watt-seconds)

# Calculate the cumulative energy used
df['Total_Energy'] = df['Incremental_Energy'].cumsum()

# Create the new time column
df['adjusted_time_ms'] = start_time_ms + (df.index - start_index) * time_per_index_ms

# Function to convert milliseconds to video time format
def ms_to_video_time(ms):
    minutes = ms // 60000
    seconds = (ms % 60000) / 1000
    return f"{int(minutes):02}:{seconds:05.2f}"

df['time_on_video'] = df['adjusted_time_ms'].apply(ms_to_video_time)

# Filter the DataFrame to include only relevant indices
df_video = df[(df.index >= start_index) & (df.index <= end_index)]

# Create the plot
fig = px.line(df_video, x='time_on_video', y='Wheel_Speed', title='Speed over Time', labels={'Wheel_Speed': 'Car Speed (m/s)'})
fig.show()


In [14]:
df_video

,TimeStamp,D1_Commanded_Torque,D1_DC_Bus_Voltage,D2_Motor_Speed,IVT_Result_I,Wheel_Speed,Power,Incremental_Energy,Total_Energy,adjusted_time_ms,time_on_video
2121,1718634549534.4570312500,29.7479000000,518.3430000000,-20.6198000000,0.3559850000,-0.1588061754,184.5223328550,1.6607009957,503.8133294930,33153005.0000000000,552:33.01
2122,1718634549543.4567871094,30.2000000000,517.7120000000,37.3778000000,0.6313780000,0.2878701764,326.8719671360,2.9418477042,506.7551771972,33153012.7182732001,552:33.01
2123,1718634549552.4562988281,30.6304000000,518.1090000000,103.3240000000,1.0990200000,0.7957637448,569.4121531800,5.1247093786,511.8798865759,33153020.4365464002,552:33.02
2124,1718634549561.4562988281,30.8000000000,517.4270000000,176.6500000000,1.5790000000,1.3604938400,817.0172330000,7.3531550970,519.2330416729,33153028.1548196003,552:33.03
2125,1718634549570.4560546875,32.1629000000,516.0540000000,240.7670000000,2.0772700000,1.8542995775,1071.9834925800,9.6478514332,528.8808931061,33153035.8730928004,552:33.04
...,...,...,...,...,...,...,...,...,...,...,...
13259,1718634649773.6071777344,12.6000000000,514.0000000000,791.8690000000,0.6473000000,6.0986860831,332.7122000000,2.9944098000,157299.6909861758,33238971.1269071996,553:58.97
13260,1718634649782.6066894531,12.6000000000,513.7970000000,797.1860000000,1.0104400000,6.1396356769,519.1610406800,4.6724493661,157304.3634355419,33238978.8451803997,553:58.98
13261,1718634649791.6066894531,12.6000000000,512.8950000000,813.3990000000,1.4310400000,6.2645022867,733.9732608000,6.6057593472,157310.9691948891,33238986.5634535998,553:58.99
13262,1718634649800.6064453125,7.5509800000,512.3050000000,840.6610000000,1.8724600000,6.4744642628,959.2706203000,8.6334355827,157319.6026304718,33238994.2817267999,553:58.99


In [15]:
df_lap = df_video[(df_video['time_on_video'] >= '552:45.014') & (df_video['time_on_video'] <= '552:51.021')]
df_lap.to_csv('AeroLap.csv')

In [16]:
# Create the plot
#fig = px.line(df, x='TimeStamp', y='Wheel_Speed', title='Wheel Speed over Time', labels={'Wheel_Speed': 'Wheel Speed (m/s)', 'TimeStamp': 'TimeStamp'})

# Show the plot
#fig.show()
df_lap

,TimeStamp,D1_Commanded_Torque,D1_DC_Bus_Voltage,D2_Motor_Speed,IVT_Result_I,Wheel_Speed,Power,Incremental_Energy,Total_Energy,adjusted_time_ms,time_on_video
3678,1718634563547.0583496094,24.7307000000,517.1750000000,310.0590000000,2.1575900000,2.3879612767,1115.8516082500,10.0426644742,8786.8193238808,33165022.3513731807,552:45.02
3679,1718634563556.0583496094,24.6000000000,516.6630000000,331.4230000000,2.0430800000,2.5524990089,1055.5838420400,9.5002545784,8796.3195784592,33165030.0696463846,552:45.03
3680,1718634563565.0581054688,24.0615000000,516.8500000000,367.4800000000,2.2095700000,2.8301968657,1142.0162545000,10.2781462905,8806.5977247497,33165037.7879195847,552:45.04
3681,1718634563574.0578613281,23.8000000000,516.3570000000,406.9170000000,2.4034600000,3.1339262490,1241.0433952200,11.1693905570,8817.7671153066,33165045.5061927848,552:45.05
3682,1718634563583.0576171875,23.8000000000,516.0930000000,441.4210000000,2.6897400000,3.3996634664,1388.1559858200,12.4934038724,8830.2605191790,33165053.2244659849,552:45.05
...,...,...,...,...,...,...,...,...,...,...,...
4451,1718634570503.8608398438,14.6000000000,514.9780000000,651.4220000000,2.1206200000,5.0170145385,1092.0726463600,9.8286538172,19269.3973709392,33170988.5765571706,552:50.99
4452,1718634570512.8603515625,14.4698000000,515.6240000000,679.8220000000,2.3606200000,5.2357409753,1217.1923268800,10.9547309419,19280.3521018811,33170996.2948303707,552:51.00
4453,1718634570521.8603515625,14.1395000000,515.1980000000,709.0410000000,2.5275300000,5.4607750512,1302.1784009400,11.7196056085,19292.0717074896,33171004.0131035708,552:51.00
4454,1718634570530.8601074219,14.0000000000,515.1940000000,737.2000000000,2.6322100000,5.6776453939,1356.0987987400,12.2048891887,19304.2765966783,33171011.7313767709,552:51.01


In [17]:
19317.1709346439 - 8786.8193238808

10530.351610763099

In [68]:
df_video = df[df['time_on_video'].apply(lambda x: int(x.split(':')[0]) * 60 + float(x.split(':')[0]) >= 0)]

In [69]:
df_video # ivt goes down 3 decimal points miliamps

,TimeStamp,D1_Commanded_Torque,IVT_Result_I,D2_Motor_Speed,D1_DC_Bus_Voltage,Wheel_Speed,Power,Incremental_Energy,Total_Energy,time_on_video
2935,1718626746430.0935058594,28.6000000000,3.9930000000,763.1350000000,510.8060000000,5.8773873002,2039.6483580000,18.3568352220,7003.0998762115,00:00.01
2936,1718626746439.0930175781,28.4424000000,5.6637200000,825.9120000000,509.3750000000,6.3608728467,2884.9573750000,25.9646163750,7029.0644925865,00:00.02
2937,1718626746448.0922851562,28.4000000000,6.8549900000,811.5080000000,507.4490000000,6.2499384948,3478.5578205100,31.3070203846,7060.3715129711,00:00.03
2938,1718626746457.0915527344,26.5089000000,6.4635100000,645.7360000000,509.8110000000,4.9732230414,3295.1684966100,29.6565164695,7090.0280294406,00:00.04
2939,1718626746466.0910644531,26.0000000000,6.0510900000,368.9340000000,514.1100000000,2.8413950431,3110.9258799000,27.9983329191,7118.0263623597,00:00.04
...,...,...,...,...,...,...,...,...,...,...
13854,1718626844694.0039062500,0.0000000000,0.0054494100,0.0000000000,14.7775000000,0.0000000000,0.0805286563,0.0007247579,269934.4714628436,01:38.28
13855,1718626844703.0031738281,0.0000000000,0.0050000000,0.0000000000,15.3225000000,0.0000000000,0.0766125000,0.0006895125,269934.4721523561,01:38.29
13856,1718626844712.0026855469,0.0000000000,0.0050000000,0.0000000000,14.4219000000,0.0000000000,0.0721095000,0.0006489855,269934.4728013416,01:38.30
13857,1718626844721.0019531250,0.0000000000,0.0052001000,0.0000000000,15.0113000000,0.0000000000,0.0780602611,0.0007025424,269934.4735038840,01:38.31


In [70]:
fig = px.line(df_video, x='time_on_video', y='Wheel_Speed', title='Speed over Time', labels={'Wheel_Speed': 'Car Speed (m/s)'})
fig.show()


In [71]:
fig = px.line(df_video, x='time_on_video', y='D1_DC_Bus_Voltage', title='Voltage over Time', labels={'D1_DC_Bus_Voltage': 'Voltage'})
fig.show()


In [72]:
fig = px.line(df_video, x='time_on_video', y='D1_Commanded_Torque', title='Torque over Time', labels={'D1_Commanded_Torque': 'Torque'})
fig.show()


In [73]:
fig = px.line(df_video, x='time_on_video', y='IVT_Result_I', title='Current over Time', labels={'IVT_Result_I': 'Current'})
fig.show()


In [75]:
df_video

,TimeStamp,D1_Commanded_Torque,IVT_Result_I,D2_Motor_Speed,D1_DC_Bus_Voltage,Wheel_Speed,Power,Incremental_Energy,Total_Energy,time_on_video
2935,1718626746430.0935058594,28.6000000000,3.9930000000,763.1350000000,510.8060000000,5.8773873002,2039.6483580000,18.3568352220,7003.0998762115,00:00.01
2936,1718626746439.0930175781,28.4424000000,5.6637200000,825.9120000000,509.3750000000,6.3608728467,2884.9573750000,25.9646163750,7029.0644925865,00:00.02
2937,1718626746448.0922851562,28.4000000000,6.8549900000,811.5080000000,507.4490000000,6.2499384948,3478.5578205100,31.3070203846,7060.3715129711,00:00.03
2938,1718626746457.0915527344,26.5089000000,6.4635100000,645.7360000000,509.8110000000,4.9732230414,3295.1684966100,29.6565164695,7090.0280294406,00:00.04
2939,1718626746466.0910644531,26.0000000000,6.0510900000,368.9340000000,514.1100000000,2.8413950431,3110.9258799000,27.9983329191,7118.0263623597,00:00.04
...,...,...,...,...,...,...,...,...,...,...
13854,1718626844694.0039062500,0.0000000000,0.0054494100,0.0000000000,14.7775000000,0.0000000000,0.0805286563,0.0007247579,269934.4714628436,01:38.28
13855,1718626844703.0031738281,0.0000000000,0.0050000000,0.0000000000,15.3225000000,0.0000000000,0.0766125000,0.0006895125,269934.4721523561,01:38.29
13856,1718626844712.0026855469,0.0000000000,0.0050000000,0.0000000000,14.4219000000,0.0000000000,0.0721095000,0.0006489855,269934.4728013416,01:38.30
13857,1718626844721.0019531250,0.0000000000,0.0052001000,0.0000000000,15.0113000000,0.0000000000,0.0780602611,0.0007025424,269934.4735038840,01:38.31


In [76]:
270161.6198422765 - 7003.0998762115

263158.519966065

In [ ]:
# 0.07309958887946250983 kw/h for skidpad